<a href="https://colab.research.google.com/github/jakewalter/intro_seismology/blob/main/tomography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Define a grid and starting velocity


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# intialize parameters and velocity perturbations
# in this case, we have 20% velocity changes that alternate from 20% to
# -20%, this is called "checkerboard" test.
X = np.arange(0.5, 2.5+1,1)

Y = np.arange(0.5, 2.5+1,1)

V0 = np.ones((3, 3)) * 6  # this is the reference homogeneous velocity

ds = np.zeros(9)
for i in range(0, 9, 2):
    ds[i] = 0.2  # ds is 20%, meaning slower velocity

for i in range(1, 9, 2):
    ds[i] = -0.2  # ds is -20%, meaning faster velocity

DS = ds.reshape(3, 3)
dv = -ds
DV = dv.reshape(3, 3)

plt.figure()
plt.imshow(DV.T, extent=(0, 3, 0, 3), cmap='jet', vmin=-0.2, vmax=0.2)
plt.colorbar(label='Velocity Perturbation')
plt.title('Velocity Perturbation Percentage')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('vel_checkboard.png')
plt.show()

In [ ]:

V = V0 * (1 + DS)  # this is the "actual" velocity
T0 = 1 / V0        # The length of each grid is 1. T0 is the expected travel
                   # time within each grid with homogeneous velocity
DT = T0 * DS       # this is the travel time anomaly contributed by each grid.
dt = DT.reshape(9, 1)



def dtoline(X, Y, s, r):
    def point_to_line(pt, v1, v2):
        a = v1 - v2
        b = pt - v2
        return np.linalg.norm(np.cross(a, b)) / np.linalg.norm(a)

    nk = 0
    d = np.zeros(len(X) * len(Y))

    for i in range(len(X)):
        for j in range(len(Y)):
            P = np.array([X[i], Y[j], 0])
            nk += 1
            v1 = np.array([s[0], s[1], 0])
            v2 = np.array([r[0], r[1], 0])
            d[nk - 1] = point_to_line(P, v1, v2)

    return d





First, consider incomplete ray coverage situation

In [ ]:
S = np.array([[0.5, 0],
              [1.5, 0],
              [2.5, 0],
              [3, 0],
              [0, 0.5],
              [0, 1.5],
              [0, 2.5],
              [1.5, 0],
              [1.5, 0],
              [0, 0]])

# R is the receiver location
R = np.array([[0.5, 3],
              [1.5, 3],
              [2.5, 3],
              [0, 3],
              [3, 0.5],
              [3, 1.5],
              [3, 2.5],
              [0, 3],
              [3, 3],
              [3, 3]])

# Start to generate ray path
plt.figure()
plt.imshow(DV.T, extent=(0, 3, 0, 3), cmap='jet', vmin=-0.2, vmax=0.2)
plt.colorbar(label='Velocity Perturbation')
plt.title('Ray Coverage with Checkerboard Test')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().set_aspect('equal', adjustable='box')

G = np.zeros((70, 9))  # Preallocate G matrix
dt_obs = np.zeros((70, 1))  # Preallocate observation matrix
ng = 0

for i in range(1, 5):
    for j in range(1, 9):
        s = S[i]
        r = R[j]
        plt.plot([s[0], r[0]], [s[1], r[1]], 'k')  # plot source-receiver pairs
        plt.plot(s[0], s[1], 'r*', markersize=20)  # plot source
        plt.plot(r[0], r[1], 'r^', markersize=20)  # plot receiver
        d = dtoline(X, Y, s, r)
        indi = np.zeros(d.shape)
        indi[d <= 0.25] = 1  # find grids along ray path
        ng += 1
        G[ng - 1, :] = indi.flatten() / V0.flatten()
        dt_obs[ng - 1, 0] = np.sum(dt[d <= 0.25]) + np.random.rand(1) * 0.01  # generate observation

plt.axis('equal')
plt.gca().invert_yaxis()
plt.savefig('ray_coverage_bad.png')
plt.show()


Now invert, notice the equation for m from the last lecture

In [ ]:
# Inversion without regularization
m = np.dot(np.dot(np.linalg.inv(np.dot(G.T,G)),G.T),dt_obs)
ds_inv = m
dv_inv = -ds_inv
dpred = G.dot(m)
residual = dt_obs - dpred
misfit = np.sqrt(np.sum(residual[~np.isnan(residual)] ** 2))
normm = np.sqrt(np.sum(m[~np.isnan(m)] ** 2))

# Display inversion results
plt.figure()
DVinv = dv_inv.reshape(3, 3)
b = plt.imshow(DVinv.T, extent=(0, 3, 0, 3), cmap='jet', vmin=-0.2, vmax=0.2)
plt.colorbar(b, label='Velocity Perturbation')
plt.title('Inversion of Velocity Perturbation')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([0, 3])
plt.ylim([0, 3])
plt.gca().invert_yaxis()
plt.savefig('vel_inversion_checkboard_singular.png')

plt.figure()
DVdiff = DVinv - DV
c = plt.imshow(DVdiff.T,extent=(0, 3, 0, 3), cmap='jet')
plt.colorbar(c, label='Difference')
plt.title('Difference between Inversion and Input')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([0, 3])
plt.ylim([0, 3])
plt.gca().invert_yaxis()
plt.savefig('vel_inversion_checkboard_singular_difference.png')



Now regularize the problem

In [ ]:

# Inversion with regularization (lambda=1)
lambda_ = 1
d_new = np.concatenate((dt_obs, np.zeros((9, 1))))
G_new = np.vstack((G, lambda_ * np.eye(9)))
m_lambda = np.dot(np.dot(np.linalg.inv(np.dot(G_new.T,G_new)),G_new.T),d_new)
ds_inv_lambda = m_lambda
dv_inv_lambda = -ds_inv_lambda

# Display inversion results with regularization
plt.figure()
DVinv_lambda = dv_inv_lambda.reshape(3, 3)
d = plt.imshow(DVinv_lambda.T, extent=(0, 3, 0, 3), cmap='jet')
plt.colorbar(d, label='Velocity Perturbation')
plt.title('Inversion of Velocity Perturbation with Regularization (lambda=1)')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([0, 3])
plt.ylim([0, 3])
plt.gca().invert_yaxis()
plt.savefig('vel_inversion_checkboard_singular_lambda=1.png')

plt.figure()
DVdiff_lambda = DVinv_lambda - DV
e = plt.imshow(DVdiff_lambda.T, extent=(0, 3, 0, 3), cmap='jet', vmin=-0.2, vmax=0.2)
plt.colorbar(e, label='Difference')
plt.title('Difference between Inversion with lambda=1 and Input')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([0, 3])
plt.ylim([0, 3])
plt.gca().invert_yaxis()
plt.savefig('vel_inversion_checkboard_singular_lambda=1_difference.png')

plt.show()

Consider a different case with better ray coverage

In [ ]:



# S is the source location
S = np.array([[0.5, 0],
              [1.5, 0],
              [2.5, 0],
              [3, 0],
              [0, 0.5],
              [0, 1.5],
              [0, 2.5],
              [1.5, 0],
              [1.5, 0],
              [0, 0]])

# R is the receiver location
R = np.array([[0.5, 3],
              [1.5, 3],
              [2.5, 3],
              [0, 3],
              [3, 0.5],
              [3, 1.5],
              [3, 2.5],
              [0, 3],
              [3, 3],
              [3, 3]])

# Start to generate ray path
plt.figure()
plt.imshow(DV.T, extent=(0, 3, 0, 3), cmap='jet', vmin=-0.2, vmax=0.2)
plt.colorbar(label='Velocity Perturbation')
plt.title('Ray Coverage with Checkerboard Test')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().set_aspect('equal', adjustable='box')


G = np.zeros((100, 9))  # Preallocate G matrix
dt_obs = np.zeros((100, 1))  # Preallocate observation matrix
ng = 0

for i in range(10):
    for j in range(10):
        s = S[i]
        r = R[j]
        plt.plot([s[0], r[0]], [s[1], r[1]], 'k')  # plot source-receiver pairs
        plt.plot(s[0], s[1], 'r*', markersize=20)  # plot source
        plt.plot(r[0], r[1], 'r^', markersize=20)  # plot receiver
        d = dtoline(X, Y, s, r)
        indi = np.zeros(d.shape)
        indi[d <= 0.25] = 1  # find grids along ray path
        ng += 1
        G[ng - 1, :] = indi.flatten() / V0.flatten()
        dt_obs[ng - 1, 0] = np.sum(dt[d <= 0.25]) + np.random.rand(1) * 0.01  # generate observation


plt.axis('equal')
plt.gca().invert_yaxis()
plt.savefig('ray_coverage_good.png')
plt.show()

Now solve

In [ ]:
# Inversion without regularization
m = np.dot(np.dot(np.linalg.inv(np.dot(G.T,G)),G.T),dt_obs)
ds_inv = m
dv_inv = -ds_inv
dpred = G.dot(m)
residual = dt_obs - dpred
misfit = np.sqrt(np.sum(residual[~np.isnan(residual)] ** 2))
normm = np.sqrt(np.sum(m[~np.isnan(m)] ** 2))

# Display inversion results
plt.figure()
DVinv = dv_inv.reshape(3, 3)
b = plt.imshow(DVinv.T, extent=(0, 3, 0, 3), cmap='jet', vmin=-0.2, vmax=0.2)
plt.colorbar(b, label='Velocity Perturbation')
plt.title('Inversion of Velocity Perturbation')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([0, 3])
plt.ylim([0, 3])
plt.gca().invert_yaxis()
plt.savefig('vel_inversion_checkboard_singular.png')

plt.figure()
DVdiff = DVinv - DV
c = plt.imshow(DVdiff.T,extent=(0, 3, 0, 3), cmap='jet')
plt.colorbar(c, label='Difference')
plt.title('Difference between Inversion and Input')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([0, 3])
plt.ylim([0, 3])
plt.gca().invert_yaxis()
plt.savefig('vel_inversion_checkboard_singular_difference.png')








Now solve with regularization, what is the difference?

In [ ]:




# Inversion with regularization (lambda=1)
lambda_ = 1
d_new = np.concatenate((dt_obs, np.zeros((9, 1))))
G_new = np.vstack((G, lambda_ * np.eye(9)))
#m_lambda = np.linalg.lstsq(G_new.T.dot(G_new), G_new.T.dot(d_new), rcond=None)[0]
m_lambda = np.dot(np.dot(np.linalg.inv(np.dot(G_new.T,G_new)),G_new.T),d_new)
ds_inv_lambda = m_lambda
dv_inv_lambda = -ds_inv_lambda

# Display inversion results with regularization
plt.figure()
DVinv_lambda = dv_inv_lambda.reshape(3, 3)
d = plt.imshow(DVinv_lambda.T, extent=(0, 3, 0, 3), cmap='jet')
plt.colorbar(d, label='Velocity Perturbation')
plt.title('Inversion of Velocity Perturbation with Regularization (lambda=1)')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([0, 3])
plt.ylim([0, 3])
plt.gca().invert_yaxis()
plt.savefig('vel_inversion_checkboard_singular_lambda=1.png')

plt.figure()
DVdiff_lambda = DVinv_lambda - DV
e = plt.imshow(DVdiff_lambda.T, extent=(0, 3, 0, 3), cmap='jet', vmin=-0.2, vmax=0.2)
plt.colorbar(e, label='Difference')
plt.title('Difference between Inversion with lambda=1 and Input')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([0, 3])
plt.ylim([0, 3])
plt.gca().invert_yaxis()
plt.savefig('vel_inversion_checkboard_singular_lambda=1_difference.png')

plt.show()


What is the effect of regularization? Let's probe that a little. Now solve for a range of lambda and examine the misfit and norm

In [ ]:




d_new = np.concatenate((dt_obs, np.zeros((9, 1))))

lambda0 = np.linspace(0.001, 10, 101)
misfit = np.zeros_like(lambda0)
normm = np.zeros_like(lambda0)
ds_inv = np.zeros((len(lambda0), 9))
dv_inv = np.zeros_like(ds_inv)

for l in range(len(lambda0)):
    lambda_ = lambda0[l]
    G_new = np.vstack((G, lambda_ * np.eye(9)))
    m_lambda = np.dot(np.dot(np.linalg.inv(np.dot(G_new.T,G_new)),G_new.T),d_new)
    ds_inv[l, :] = m_lambda.T
    dv_inv[l, :] = -m_lambda.T
    dpred = G.dot(m_lambda)
    residual = dt_obs - dpred
    misfit[l] = np.sqrt(np.sum(residual ** 2))
    normm[l] = np.sqrt(np.sum(m_lambda ** 2))


x = np.sqrt((normm / np.max(normm))**2 + (misfit / np.max(misfit))**2)
nbest = np.where(x == np.min(x))[0]
print(lambda0[nbest[0]])

plt.figure()
#plot lambda vs misfit
plt.plot()


plt.figure()
#plot lambda vs norm
plt.plot()

plt.figure()
#plot misfit vs norm

plt.figure()
#plot lambda0 vs x



Plot the model at optimal lambda, then plot model with smallest lambda and model with largest lambda.

Show the three figures side-by-side, and compare the difference.

Note that in the sample code above, we saved the model for each lambda in dv_inv. To plot the model for specific lambda values, you need to find the index number for the lambda requested, then get the model:
DVinv = np.reshape(dv_inv[nbest],(3,3))


Then, you follow the code above to plot the velocity images.


np.reshape(dv_inv[nbest],(3,3))